In [1]:
!pip install transformers torch datasets evaluate rouge_score pip accelerate
!pip install transformers --upgrade
!nvidia-smi
!pip install ipywidgets
!pip install python-dotenv!python3 fine-tuned_ByT5.py


Mon Oct 13 12:08:02 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.274.02             Driver Version: 535.274.02   CUDA Version: 12.6     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:0F:00.0 Off |                   On |
| N/A   28C    P0              48W / 400W |                  N/A |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")

# print("HF_TOKEN: ", HF_TOKEN)

# source: https://huggingface.co/docs/transformers/v4.28.1/tasks/summarization
from huggingface_hub import login

login(HF_TOKEN)
LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","mul","deu_eng"]
LANGS = ["grc"]

PATH_JSON_DATA = "../preprocessing_to_json/data/"

for lang in LANGS:
    from datasets import load_dataset
    # dataset= dataset.train_test_split(test_size=0.2) #train_size=10000,
    dataset = load_dataset("json", data_files={"train": PATH_JSON_DATA + lang + "_train.json", "validation": PATH_JSON_DATA + lang + "_eval.json"})

    print(dataset["train"][0])

    from transformers import AutoTokenizer

    checkpoint = "google/byt5-small"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    # -

    import torch

    from transformers import (
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        AutoModelForSeq2SeqLM,
        Seq2SeqTrainingArguments,
        Seq2SeqTrainer
    )
    from datasets import load_dataset

    # prefix = "summarize: "


    def preprocess_function(examples):
        # inputs = [prefix + doc for doc in examples["text"]]
        inputs = examples["text"]
        model_inputs = tokenizer(inputs)

        labels = tokenizer(text_target=examples["summary"]) # same length irrelevant here, so no truncation or padding

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs


    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    print(tokenized_dataset)
    sample = tokenized_dataset["train"][0]
    print(sample["labels"])

    from transformers import DataCollatorForSeq2Seq

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

    # import evaluate

    # rouge = evaluate.load("rouge")

    # import numpy as np


    # def compute_metrics(eval_pred):
    #     predictions, labels = eval_pred
    #     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    #     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    #     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    #     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    #     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    #     result["gen_len"] = np.mean(prediction_lens)

    #     return {k: round(v, 4) for k, v in result.items()}

    # import numpy as np
    # import evaluate

    # metric = evaluate.load("accuracy")

    # def compute_metrics(eval_pred):
    #     predictions, labels = eval_pred # TODO: labels vs title?

    #     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    #     labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # TODO: correct?

    #     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #     return metric.compute(predictions=predictions, references=labels)

    from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    training_args = Seq2SeqTrainingArguments(
        output_dir="byt5-small/"+lang,
        eval_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=4,
        predict_with_generate=True,
        # save_strategy="epoch",
        fp16=False,# did not work with True #change to bf16=True for XPU 
        # bf16 = True, # because Nvidia Ampere A100 supports bf16
        push_to_hub=True,
        warmup_steps = 500,
        # load_best_model_at_end=True, # otherwise not model with minimum loss during training, like https://huggingface.co/docs/transformers/tasks/sequence_classification
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.push_to_hub()


{'text': 'Inflect Βάκτριος using N;ACC(DU)', 'summary': 'Βακτρίω'}
DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})
[209, 149, 209, 180, 209, 189, 210, 135, 210, 132, 209, 178, 210, 140, 1]


/tmp/ipykernel_359268/3423468971.py:126: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


In [41]:
LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","mul","deu_eng"]
LANGS = ["grc"]

for lang in LANGS:
    import torch
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
    PRETRAINED_MODEL = "google/byt5-small"
    FINETUNED_MODEL ="livles/byt5-small/"+lang
    FINETUNED_MODEL ="./byt5-small/"+lang
    MODEL = FINETUNED_MODEL
    print(MODEL)
    tokenizer = AutoTokenizer.from_pretrained(
        FINETUNED_MODEL,
    )
    model = AutoModelForSeq2SeqLM.from_pretrained(
        FINETUNED_MODEL,
    dtype=torch.float16,
    device_map="auto"
    )

    from datasets import load_dataset
    OUT_DIR = "output_byt5_small/"
    PATH = "../preprocessing_to_json/data/" + lang + "_test.json"
    PATH_ORIG = "../2023InflectionST/part1/data/" + lang + ".tst"
    LEN = 1000
    with open (OUT_DIR + lang + "_byt5-small.out","w") as out_file, open (PATH_ORIG, "r") as test_file:
        COUNTER = 0
        dataset = load_dataset("json",data_files={"test":PATH})
        covered_test_lines = dataset["test"]["text"]


        # preds = map (generate,covered_test_lines)
        for line in covered_test_lines:
            input_ids = tokenizer(line, return_tensors="pt").to(model.device)
            output = model.generate(
                **input_ids,
                max_new_tokens=50,
                num_beams=4,
                #early_stopping=True
            )
            output_string = (tokenizer.decode(output[0], skip_special_tokens=True))
            lemma, features, ref = test_file.readline().strip().split("\t")

            # evaluate
            if (ref == output_string):
                COUNTER += 1
                print(ref,output_string)
            else:
                print("ref:",ref,"pred:",output_string)
            
            # write
            out_file.write(lemma + "\t" +features+ "\t" + output_string + "\n")
            out_file.flush()

        accuracy = COUNTER / 1000
        out_file.write("accuracy:"+ str(accuracy))
        print(accuracy)


./byt5-small/grc
Βῐσᾰ́λτᾱ Βῐσᾰ́λτᾱ
ref: Βῐσᾰ́λτᾱς pred: Βῐσᾰ́λταις
ref: Βῐσᾰ́λτην pred: Βῐσᾰ́λτης
Βῐσᾰ́λταιν Βῐσᾰ́λταιν
Βῐσᾰ́λταις Βῐσᾰ́λταις
ref: Βῐσᾰ́λτῃ pred: Βῐσᾰ́λτῳ
Βῐσᾰ́λταιν Βῐσᾰ́λταιν
ref: Βῐσᾰλτῶν pred: Βῐσᾰ́λταις
Βῐσᾰ́λτου Βῐσᾰ́λτου
Βῐσᾰ́λτᾱ Βῐσᾰ́λτᾱ
Βῐσᾰ́λται Βῐσᾰ́λται
Βῐσᾰ́λτης Βῐσᾰ́λτης
Βῐσᾰ́λτᾱ Βῐσᾰ́λτᾱ
Βῐσᾰ́λται Βῐσᾰ́λται
ref: Βῐσᾰ́λτᾰ pred: Βῐσᾰ́λτη
ref: Μασσαλιώτᾱ pred: Μᾰσσᾰλῐώτᾱ
ref: Μασσαλιώτᾱς pred: Μᾰσσᾰλῐώταις
ref: Μασσαλιώτην pred: Μασσαλιώτης
Μασσαλιώταιν Μασσαλιώταιν
Μασσαλιώταις Μασσαλιώταις
ref: Μασσαλιώτῃ pred: Μᾰσσᾰλῐώτῳ
Μασσαλιώταιν Μασσαλιώταιν
ref: Μασσαλιωτῶν pred: Μασσαλιώταις
Μασσαλιώτου Μασσαλιώτου
ref: Μασσαλιώτᾱ pred: Μᾰσσᾱλῐώτω
ref: Μασσαλιῶται pred: Μασσαλιώται
Μασσαλιώτης Μασσαλιώτης
ref: Μασσαλιώτᾱ pred: Μᾰσσᾱλῐώτω
ref: Μασσαλιῶται pred: Μασσαλιώται
ref: Μασσαλιῶτᾰ pred: Μασσαλιώτη
Μιθριακώ Μιθριακώ
Μιθριακώ Μιθριακώ
ref: Μιθριακᾱ́ς pred: Μιθριακαί
Μιθριακούς Μιθριακούς
Μιθριακᾰ́ Μιθριακᾰ́
Μιθριακήν Μιθριακήν
Μιθριακαῖν Μιθριακαῖν
Μιθριακοῖν 